In [1]:
import pandas as pd
import pymongo

In [2]:
client = pymongo.MongoClient('localhost')

In [3]:
db = client['desafio']
aeronaves = db['aeronaves']
ocorrencias = db['ocorrencias']
ocorrencias_tipo = db['ocorrencias_tipo']

In [4]:
df_ocorrencia = pd.read_csv('../data/ocorrencia.csv', encoding='ISO-8859-1', sep=';')
df_ocorrencia.head()

,id_ocorrencia,classificacao,cidade,uf,pais,data,num_recomendacoes
0,39115,ACIDENTE,CORRENTINA,BA,BRASIL,2010-02-07T17:40:00Z,2
1,39155,INCIDENTE,BELO HORIZONTE,MG,BRASIL,2010-02-05T12:55:00Z,0
2,39156,INCIDENTE GRAVE,CASCAVEL,PR,BRASIL,2010-01-10T23:15:00Z,2
3,39158,INCIDENTE,BELEM,PA,BRASIL,2010-01-28T16:00:00Z,0
4,39176,INCIDENTE,SAO LUIS,MA,BRASIL,2010-02-05T09:30:00Z,0


In [5]:
df_ocorrencia.dtypes

id_ocorrencia         int64
classificacao        object
cidade               object
uf                   object
pais                 object
data                 object
num_recomendacoes     int64
dtype: object

In [6]:
df_ocorrencia['data'] = df_ocorrencia['data'].str.replace('NULL', '')
df_ocorrencia['data'] = df_ocorrencia['data'].str.replace('T', ' ').str.replace('Z', '')

In [7]:
dtypes_ocorr_dict = {
    'id_ocorrencia': 'int',
    'classificacao': 'str',
    'cidade': 'str',
    'uf': 'str',
    'pais': 'str',
    'data': 'datetime64',
    'num_recomendacoes': 'int'
}

df_ocorrencia = df_ocorrencia.astype(dtypes_ocorr_dict)
    

In [8]:
df_ocorrencia.dtypes

id_ocorrencia                 int64
classificacao                object
cidade                       object
uf                           object
pais                         object
data                 datetime64[ns]
num_recomendacoes             int64
dtype: object

In [9]:
validator_ocorrencia={
        '$jsonSchema': {
            'bsonType': 'object',
            'properties': {
                'id_ocorrencia': {
                    'bsonType': 'int',
                    'description': 'id of the occurence, is not required'
                },
                'classificacao': {
                    'bsonType': 'string',
                    'description': 'classification of the occurence, is not required'
                },
                'cidade': {
                    'bsonType': 'string',
                    'description': 'city origin of the occurence, is not required'
                },
                'uf': {
                    'bsonType': 'string',
                    'description': 'UF origin of the occurence, is not required'
                },
                'pais': {
                    'bsonType': 'string',
                    'description': 'Country of occurence, is not required'
                },
                'data': {
                    'bsonType': 'date',
                    'description': 'data of the occurence, is not required'
                },
                'num_recomendacoes': {
                    'bsonType': 'int',
                    'description': 'num of recomendation for the occurence, is not required'
                }
            }
        }
    }

In [10]:
db.create_collection(
    'ocorrencia',
    validator=validator_ocorrencia
)

Collection(Database(MongoClient(host=['localhost:27017'], document_class=dict, tz_aware=False, connect=True), 'desafio'), 'ocorrencia')

In [11]:
df_ocorrencia_tipo = pd.read_csv("../data/ocorrencia_tipo.csv", encoding='utf-8', sep=';')
df_ocorrencia_tipo.head()

,id_ocorrencia_t,tipo
0,39115,PANE SECA
1,39155,VAZAMENTO DE COMBUSTIVEL
2,39156,FOGO EM VOO
3,39158,SUPERAQUECIMENTO
4,39176,FALHA OU MAU FUNCIONAMENTO DE SISTEMA / COMPON...


In [12]:
validator_ocorrencia_tipo = {
    '$jsonSchema': {
        'bsonType': 'object',
        'properties': {
            'id_ocorrencia_t': {
                'bsonType': 'int',
                'description': 'id of the occurence, is not required'
            },
            'tipo': {
                'bsonType': 'string',
                'description': 'type of occurence, is not required'
            }
        }
    }
}


In [13]:
db.create_collection(
    'ocorrencia_tipo',
    validator=validator_ocorrencia_tipo
)

Collection(Database(MongoClient(host=['localhost:27017'], document_class=dict, tz_aware=False, connect=True), 'desafio'), 'ocorrencia_tipo')

In [14]:
df_ocorrencia_tipo.dtypes

id_ocorrencia_t     int64
tipo               object
dtype: object

In [15]:
dtypes_ocorrencia_tipo_dict = {
    'id_ocorrencia_t': 'int',
    'tipo': 'str'
}

df_ocorrencia_tipo = df_ocorrencia_tipo.astype(dtypes_ocorrencia_tipo_dict)

In [16]:
df_aeronave = pd.read_csv("../data/aeronave.csv", encoding='iso-8859-1', sep=';')
df_aeronave.head()

,id_ocorrencia_a,matricula,operador_categoria,tipo_veiculo,fabricante,modelo,motor_tipo,motor_quantidade,assentos,ano_fabricacao,pais_fabricante,registro_segmento,voo_origem,voo_destino,fase_operacao
0,39115,PTNQX,***,AVIAO,NEIVA INDUSTRIA AERONAUTICA,EMB-711A,PISTAO,MONOMOTOR,4,1979,BRASIL,PARTICULAR,BRIGADEIRO ARARIPE MACEDO,CORRENTINA,DECOLAGEM
1,39155,PTLVI,***,AVIAO,BEECH AIRCRAFT,C90,TURBOELICE,BIMOTOR,8,1979,BRASIL,PARTICULAR,FORA DE AERODROMO,FORA DE AERODROMO,DECOLAGEM
2,39156,PPPTO,***,AVIAO,AEROSPATIALE AND ALENIA,ATR-72-212A,TURBOELICE,BIMOTOR,73,2008,BRASIL,REGULAR,AFONSO PENA,ADALBERTO MENDES DA SILVA,ARREMETIDA NO AR
3,39158,PRLGJ,REGULAR,AVIAO,BOEING COMPANY,757-225,JATO,BIMOTOR,5,1984,BRASIL,REGULAR,FORA DE AERODROMO,FORA DE AERODROMO,SUBIDA
4,39176,PRMAA,REGULAR,AVIAO,AIRBUS INDUSTRIE,A320-232,JATO,BIMOTOR,184,2001,BRASIL,REGULAR,FORA DE AERODROMO,FORA DE AERODROMO,SUBIDA


In [17]:
validator_aeronave = {
    '$jsonSchema': {
        'bsonType': 'object',
        'required': ["assentos", "ano_favricacao"],
        'properties': {
            'id_ocorrencia_a': {
                'bsonType': 'int',
                'description': 'id of the occurence, is not required'
            },
            'matricula': {
                'bsonType': 'string',
                'description': 'matricula of the occurence, is not required'
            },
            'operador_categoria': {
                'bsonType': 'string',
                'description': 'operador_categoria of the occurence, is not required'
            },
            'tipo_veiculo': {
                'bsonType': 'string',
                'description': 'tipo_veiculo of the occurence, is not required'
            },
            'fabricante': {
                'bsonType': 'string',
                'description': 'fabricante of the occurence, is not required'
            },
            'modelo': {
                'bsonType': 'string',
                'description': 'modelo of the occurence, is not required'
            },
            'motor_tipo': {
                'bsonType': 'string',
                'description': 'motor_tipo of the occurence, is not required'
            },
            'motor_quantidade': {
                'bsonType': 'int',
                'description': 'motor_quantidade of the occurence, is not required'
            },
            'assentos': {
                'bsonType': 'int',
                'minimum': 1,
                'maximum': 1000,
                'description': 'assentos of the occurence, must be an integer in [ 1, 1000 ] and is required'
            },
            'ano_fabricacao': {
                'bsonType': 'int',
                'minimum': 1950,
                'maximum': 2030,
                'description': 'ano_fabricacao of the occurence, must be an integer in [ 1950, 2030 ] and is required'
            },
            'pais_fabricante': {
                'bsonType': 'string',
                'description': 'pais_fabricante of the occurence, is not required'
            },
            'registro_segmento': {
                'bsonType': 'string',
                'description': 'registro_segmento of the occurence, is not required'
            },
            'voo_origem': {
                'bsonType': 'string',
                'description': 'voo_origem of the occurence, is not required'
            },
            'voo_destino': {
                'bsonType': 'string',
                'description': 'voo_destino of the occurence, is not required'
            },
            'fase_operacao': {
                'bsonType': 'string',
                'description': 'fase_operacao of the occurence, is not required'
            }
        }
    }
}

In [18]:
db.create_collection(
    'aeronave',
    validator=validator_aeronave
)

Collection(Database(MongoClient(host=['localhost:27017'], document_class=dict, tz_aware=False, connect=True), 'desafio'), 'aeronave')

In [19]:
df_aeronave.dtypes

id_ocorrencia_a        int64
matricula             object
operador_categoria    object
tipo_veiculo          object
fabricante            object
modelo                object
motor_tipo            object
motor_quantidade      object
assentos               int64
ano_fabricacao         int64
pais_fabricante       object
registro_segmento     object
voo_origem            object
voo_destino           object
fase_operacao         object
dtype: object

In [20]:
dtypes_aeronave = {
    'id_ocorrencia_a': 'int',
    'matricula': 'str',
    'operador_categoria': 'str',
    'tipo_veiculo': 'str',
    'fabricante': 'str',
    'modelo': 'str',
    'motor_tipo': 'str',
    'motor_quantidade': 'str',
    'assentos': 'int',
    'ano_fabricacao': 'int',
    'pais_fabricante': 'str',
    'registro_segmento': 'str',
    'voo_origem': 'str',
    'voo_destino': 'str',
    'fase_operacao': 'str'
}

df_aeronave = df_aeronave.astype(dtypes_aeronave)    
    

In [21]:
df_aeronave.dtypes

id_ocorrencia_a        int64
matricula             object
operador_categoria    object
tipo_veiculo          object
fabricante            object
modelo                object
motor_tipo            object
motor_quantidade      object
assentos               int64
ano_fabricacao         int64
pais_fabricante       object
registro_segmento     object
voo_origem            object
voo_destino           object
fase_operacao         object
dtype: object

In [22]:
def documents_inserted_validation(inserted, df):
    inserted_rows = inserted.inserted_ids

    print("Inserted Count: ", len(inserted_rows))
    print("Df Count: ", len(df))

    if len(inserted_rows) == len(df):
        print("All documents were inserted")
    else:
        print(f'{len(df) - len(inserted_rows)} documents were not inserted')
        print('as the following:')
        print(df[~df['id_ocorrencia_a'].isin(inserted_rows)])

In [23]:
insert_aeronaves_coll  =  aeronaves.insert_many(df_aeronave.to_dict('records'))
documents_inserted_validation(insert_aeronaves_coll, df_aeronave)

Inserted Count:  5213
Df Count:  5213
All documents were inserted


In [24]:
insert_ocorrencias_coll = ocorrencias.insert_many(df_ocorrencia.to_dict('records'))
documents_inserted_validation(insert_ocorrencias_coll, df_ocorrencia)

Inserted Count:  5155
Df Count:  5155
All documents were inserted


In [25]:
insert_ocorrencias_tipo_coll = ocorrencias_tipo.insert_many(df_ocorrencia_tipo.to_dict('records'))
documents_inserted_validation(insert_ocorrencias_tipo_coll, df_ocorrencia_tipo)

Inserted Count:  5244
Df Count:  5244
All documents were inserted
